# Series de tiempo y redes neuronales.

Hay varias maneras de usar redes neuronales para predecir series de tiempo. Nuestro objetivo será llegar a transfomers, pero iremos explorando las ideas poco a poco.

Lo que vamos a hacer aquí es redes neuronales "fully connected", pero si lo piensas bien, estamos usando la idea de las redes neuronales convolucionales. Quedará claro por qué a lo largo del video.

Seguiremos usando la serie de tiempo de las temperaturas.

In [1]:
import numpy as np
import pandas as pd
import fastai.tabular.all as ft
import torch
import torch.nn as nn
import torch.nn.functional as F
import random

In [2]:
torch.cuda.set_device(1)

In [3]:
torch.cuda.current_device()

1

In [4]:
df = pd.read_csv("daily-min-temperatures.csv",index_col='Date',parse_dates=True)

In [ ]:
df.plot()

## ¿Cómo usar redes neuronales?

Hay muchas cosas que podemos hacer para usar redes neuronales en series de tiempo. Vamos a empezar por lo más más sencillo y poco a poco iremos agregando cosas. El plan será:

1. Que la red tome exclusivamente la temperatura del día anterior para predecir la temperatura del día siguiente.
2. Convolucionales: Agregarle las últimas $k$ temperaturas para que pueda hacer la predicción.
3. Agregarle información del mes y del año.
4. Agregarle por ejemplo la EWMA.
5. Agregarle la desviación estándar, y otra información.
6. ¿Por qué redes recurrentes?


### Cargando los datos

Recordemos que fastai puede cargar datos "tabulares" (e.g. un dataframe de pandas).

In [ ]:
df.head()

Queremos predecir la temperatura cada día basados en la temperatura anterior, así que debemos crear la columna "ultima_temp".

In [ ]:
df['ultima_temp'] = df['Temp'].shift(1)

In [ ]:
df = df.dropna()

In [ ]:
df.head()

In [ ]:
def error_l1(df,colA,colB):
    df = df.iloc[-365:]
    return np.mean(np.abs(df[colA]-df[colB]))
def error_rmse(df,colA,colB):
    df = df.iloc[-365:]
    return np.sqrt(np.mean(np.square(df[colA]-df[colB])))

In [ ]:
error_l1(df,'Temp','ultima_temp')

In [ ]:
error_rmse(df,'Temp','ultima_temp')

In [ ]:
alumnos = ["Miguel", "Paola", "Arnold", "Diego", "Gerardo", "Erika", "Hannia,", "Leo Grande", "Dani", "Leo chico"]

In [ ]:
random.choice(alumnos)

In [ ]:
def cargar_datos(df):
    n = len(df)
    v = n-365
    src = ft.TabularPandas(df,
                     cont_names = ['ultima_temp'],
                     y_names = ['Temp'],
                     splits = (list(range(v)),list(range(v,n)))
                          )
    return src.dataloaders(bs = 256)

In [ ]:
dls = cargar_datos(df)

In [ ]:
def cargar_datos(df):
    X = ft.range_of(df)
    valid_cut = len(df) - 365
    src = ft.TabularPandas(df,
                       cont_names=list(df.columns[1:]),
                       y_names=['Temp'],
                       splits=(X[:valid_cut],X[valid_cut:]))
    return src.dataloaders(bs=1024)

In [ ]:
dls = cargar_datos(df)

In [ ]:
x_cat, x_cont, y = dls.one_batch()

No se preocupen por el tensor vacío. Lo que ocurre es que fastai divide en datos categóricos y datos contínuos. Sin embargo, ahora no tenemos ningún dato categórico (luego tendremos), así que simplemente me da vacío en los categóricos. No usaremos eso por ahora.

In [17]:
def rmse_loss(yp,y):
    return torch.sqrt(F.mse_loss(yp,y))

In [ ]:
learn=ft.tabular_learner(dls,layers=[],metrics=[F.l1_loss,rmse_loss])

In [ ]:
learn.model

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(50,1e-1)

Más o menos tiene el mismo error que la baseline (poquito mejor), lo cual es bueno! Vamos a ver si podemos hacerlo mejor con más capas.

In [ ]:
learn=ft.tabular_learner(dls,layers=[128,128],metrics=[F.l1_loss,rmse_loss])

In [ ]:
learn.lr_find()

In [ ]:
learn.model

In [ ]:
learn.fit_one_cycle(50,3e-2)

Un poquito mejor, pero... no mucho. Pues es que piénsenlo: nada más le dimos la temperatura anterior. Cómo le va a hacer para hacerlo mejor?

## Redes convolucionales.

Si en vez de darle la temperatura anterior le damos las $k$ anteriores, ¿podremos mejorarlo?

In [ ]:
import random

In [ ]:
df = pd.read_csv("daily-min-temperatures.csv",index_col='Date',parse_dates=True)

In [ ]:
df

In [5]:
k = 8
for i in range(1,k+1) :
    df[f'temp_anterior{i}'] = df['Temp'].shift(i)

In [ ]:
random.choice(alumnos)

In [ ]:
def cargar_datos(df):
    n = len(df)
    v = n-365
    src = ft.TabularPandas(df,
                     cont_names = [f'temp_anterior{i}' for i in range(1,k+1)],
                     y_names = ['Temp'],
                     splits = (list(range(v)),list(range(v,n)))
                          )
    return src.dataloaders(bs = 256)

In [ ]:
random.choice(alumnos)

In [ ]:
dls = cargar_datos(df)

In [ ]:
learn=ft.tabular_learner(dls,
                         layers=[128],
                         opt_func=ft.ranger,
                         metrics=[F.l1_loss,rmse_loss])

In [ ]:
learn.model

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(50,7e-2,div=1,pct_start=0.7)

In [ ]:
random.choice(alumnos)

In [ ]:
df

In [ ]:
fecha = df.index.year

In [ ]:
fecha

In [ ]:
df=df.copy()

In [6]:
df['año'] = df.index.year
df['mes'] = df.index.month
df['dia'] = df.index.day

In [7]:
df

,Temp,temp_anterior1,temp_anterior2,temp_anterior3,temp_anterior4,temp_anterior5,temp_anterior6,temp_anterior7,temp_anterior8,año,mes,dia
Date,,,,,,,,,,,,
1981-01-01,20.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1981,1,1
1981-01-02,17.9,20.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1981,1,2
1981-01-03,18.8,17.9,20.7,NaN,NaN,NaN,NaN,NaN,NaN,1981,1,3
1981-01-04,14.6,18.8,17.9,20.7,NaN,NaN,NaN,NaN,NaN,1981,1,4
1981-01-05,15.8,14.6,18.8,17.9,20.7,NaN,NaN,NaN,NaN,1981,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1990-12-27,14.0,14.6,12.9,10.0,13.9,13.2,13.1,15.4,14.7,1990,12,27
1990-12-28,13.6,14.0,14.6,12.9,10.0,13.9,13.2,13.1,15.4,1990,12,28
1990-12-29,13.5,13.6,14.0,14.6,12.9,10.0,13.9,13.2,13.1,1990,12,29


In [8]:
k = 8
for i in range(1,k+1):
    df[f'ultima_{i}'] = df['Temp'].shift(i)

In [9]:
df['sin_mes'] = np.sin(2*np.pi*df['mes']/12)

In [10]:
df['cos_mes'] = np.cos(2*np.pi*df['mes']/12)

In [11]:
for a in np.linspace(0.1, 0.9, 9): 
    df[f"ewma_{a:.1f}"] = df["Temp"].ewm(alpha=a).mean().shift(1) 

In [12]:
df

,Temp,temp_anterior1,temp_anterior2,temp_anterior3,temp_anterior4,temp_anterior5,temp_anterior6,temp_anterior7,temp_anterior8,año,...,cos_mes,ewma_0.1,ewma_0.2,ewma_0.3,ewma_0.4,ewma_0.5,ewma_0.6,ewma_0.7,ewma_0.8,ewma_0.9
Date,,,,,,,,,,,,,,,,,,,,,
1981-01-01,20.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1981,...,0.866025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1981-01-02,17.9,20.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1981,...,0.866025,20.700000,20.700000,20.700000,20.700000,20.700000,20.700000,20.700000,20.700000,20.700000
1981-01-03,18.8,17.9,20.7,NaN,NaN,NaN,NaN,NaN,NaN,1981,...,0.866025,19.226316,19.144444,19.052941,18.950000,18.833333,18.700000,18.546154,18.366667,18.154545
1981-01-04,14.6,18.8,17.9,20.7,NaN,NaN,NaN,NaN,NaN,1981,...,0.866025,19.069004,19.003279,18.937443,18.873469,18.814286,18.764103,18.728777,18.716129,18.736036
1981-01-05,15.8,14.6,18.8,17.9,20.7,NaN,NaN,NaN,NaN,1981,...,0.866025,17.769497,17.511653,17.225069,16.909559,16.566667,16.200000,15.815032,15.417949,15.013231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1990-12-27,14.0,14.6,12.9,10.0,13.9,13.2,13.1,15.4,14.7,1990,...,1.000000,13.688833,13.448916,13.346300,13.371937,13.498774,13.695645,13.930007,14.174191,14.404831
1990-12-28,13.6,14.0,14.6,12.9,10.0,13.9,13.2,13.1,15.4,1990,...,1.000000,13.719949,13.559133,13.542410,13.623162,13.749387,13.878258,13.979002,14.034838,14.040483
1990-12-29,13.5,13.6,14.0,14.6,12.9,10.0,13.9,13.2,13.1,1990,...,1.000000,13.707955,13.567306,13.559687,13.613897,13.674694,13.711303,13.713701,13.686968,13.644048


In [13]:
df = df.dropna()

In [14]:
def cargar_datos(df):
    n = len(df)
    v = n-365
    cont_names = [f'temp_anterior{i}' for i in range(1,k+1)]
    cont_names += ['sin_mes','cos_mes','año']
    cont_names += [c for c in df.columns if "ewma" in c]
    src = ft.TabularPandas(df,
                     cont_names = cont_names,
                     y_names = ['Temp'],
                     splits = (list(range(v)),list(range(v,n)))
                          )
    return src.dataloaders(bs = 256)

In [15]:
dls = cargar_datos(df)

In [18]:
learn=ft.tabular_learner(dls,
                         layers=[128],
                         opt_func=ft.ranger,
                         metrics=[F.l1_loss,rmse_loss])

In [19]:
learn.fit_one_cycle(50,3e-2,div=1,pct_start=0.7)

epoch,train_loss,valid_loss,l1_loss,rmse_loss,time
0,89.189774,7590.398438,87.090240,87.122749,00:00
1,78.902626,309.442108,17.435600,17.589119,00:00
2,67.832069,69.562180,8.004925,8.336593,00:00
3,53.643372,26.097044,4.585136,5.104037,00:00
4,39.808037,5.314179,1.818446,2.299230,00:00
5,30.819460,7.704399,2.236255,2.774560,00:00
6,24.530983,5.570164,1.848741,2.356559,00:00
7,19.804720,5.016495,1.725960,2.233128,00:00
8,16.412106,4.936713,1.722957,2.216110,00:00
9,13.822941,4.822562,1.703071,2.191320,00:00


In [ ]:
def cargar_datos(df):
    X = ft.range_of(df)
    valid_cut = len(df) - 365
    cat_names = ['Month']
    cont_names = [ c for c in df.columns[1:] if c not in cat_names]
    src = ft.TabularPandas(df,
                       cont_names=cont_names,
                       cat_names = cat_names,
                       y_names=['Temp'],
                       procs=[ft.Categorify,ft.Normalize],
                       splits=(X[:valid_cut],X[valid_cut:]))
    return src.dataloaders(bs=1024)

In [ ]:
dls = cargar_datos(df)

In [ ]:
learn=ft.tabular_learner(dls,layers=[64],metrics=[rmse_loss,F.l1_loss])

In [ ]:
learn.fit_one_cycle(50,3e-2)

Mejor

# Modelo

Estamos usando una red neuronal "fully connected":

In [ ]:
learn.model

- embeds es vacío porque no tenemos variables categóricas.
- emb_drop es el embedding dropout. Podemos agregarle si queremos.
- bn_cont es la capa batchnorm que le aplicamos a las cariables continuas.
- layers son las capas que le dijimos.

Podemos modificarla como normalmente hacemos. Por ejemplo:

In [ ]:
learn.model.layers[0][1] = nn.CELU()

In [ ]:
learn.model.layers[1][1] = nn.CELU()

In [ ]:
learn.fit_one_cycle(50,3e-2)

## Tareas

Ejercicio 1: Agrega información de año y mes, y agrega el EWMA también. Repite todo a ver si llegas a menos.

Ejercicio 2: Repite esto con alguna otra serie de tiempo.

Ejercicio 3: Agrega la información producida por Hodrick-Prescott o Holt Winters. Vamos a ver quién llega a menos rmse :)